This notebook demonstrates downloading HLS data, which will be used in the subsequent notebook to create a NDVI layer as model input. Users are required to have a [NASA Earthdata account](https://urs.earthdata.nasa.gov/), with the credentials stored in a `.netrc` file in the user's home directory in the following format:

`
machine urs.earthdata.nasa.gov login {user_name} password {password}
`

with `{username}` and `{password}` replaced with user values

In [27]:
# GIS imports
from osgeo import gdal
import rasterio
from datetime import datetime
from rasterio.warp import transform_bounds
from rasterio.crs import CRS
from rasterio.merge import merge

# Earthaccess imports to query and retrieve data
import earthaccess

# Misc imports
from pathlib import Path
import shutil

# GDAL setup for accessing cloud data
gdal.SetConfigOption('GDAL_HTTP_COOKIEFILE','~/cookies.txt')
gdal.SetConfigOption('GDAL_HTTP_COOKIEJAR', '~/cookies.txt')
gdal.SetConfigOption('GDAL_DISABLE_READDIR_ON_OPEN','EMPTY_DIR')
gdal.SetConfigOption('CPL_VSIL_CURL_ALLOWED_EXTENSIONS','TIF, TIFF, XML, xml')

# Ensure that the .netrc file has credentials set appropriately
auth = earthaccess.login(strategy="netrc")

In [29]:
# We use the bounds and CRS of an input stereo image (corresponding to the initial DSM) to query NASA Earthdata for HLS data
stereo_data_path = Path('../data/mt_baker/WV01_20150911_1020010042D39D00_1020010043455300/original_rasters')
stereo_img_path = stereo_data_path / '20150911_2050_1020010042D39D00_1020010043455300_1.0m-DEM_trans_dx+2.40m_dy+1.51m_dz-0.36m__ASP_wt_avg_extrapolation_search_rad25_num_pass_3.tif'

assert stereo_img_path.exists(), "Specified file does not exist!" # check that the file is accessible

In [3]:
# Set up search parameters (bounds, temporal search window)
with rasterio.open(stereo_img_path) as ds:
    crs = ds.profile['crs']
    bounds = ds.bounds

crs = CRS.from_epsg(32610)
assert crs is not None, "CRS cannot be None. Manually set using EPSG code"

bounds_4326 = transform_bounds(crs, CRS.from_epsg(4326), *bounds)

# specify search window corresponding to when stereo images were acquired
start_date = datetime(year=2015, month=9, day=4)
end_date = datetime(year=2015, month=9, day=18)

time_window = (start_date.strftime("%Y-%m-%dT00:00:00"), end_date.strftime("%Y-%m-%dT23:59:59"))

In [4]:
# Query NASA Earthdata
results = earthaccess.search_data(
    short_name=['HLSL30','HLSS30'],
    bounding_box=bounds_4326,
    temporal=time_window,
    count=100
)

In [16]:
# Setup folder structure for downloads. HLS files are stored in the ../data/common_files folder
download_tmp_path = Path("../data/common_files/tmp")
download_tmp_path.mkdir(parents=True, exist_ok=True)
final_download_path = Path("../data/common_files")

In [13]:
# Download the data
files = earthaccess.download(results, download_tmp_path)

QUEUEING TASKS | :   0%|          | 0/30 [00:00<?, ?it/s]

PROCESSING TASKS | :   0%|          | 0/30 [00:00<?, ?it/s]

COLLECTING RESULTS | :   0%|          | 0/30 [00:00<?, ?it/s]

In [24]:
# merge tiles if more than one result is returned

hls_red_band_files = list(download_tmp_path.glob("*B04*.tif"))
hls_nir_band_files = list(download_tmp_path.glob("*B05*.tif"))

if len(results) > 0:
    red_merged, transform = merge(hls_red_band_files)
    nir_merged, transform = merge(hls_nir_band_files)

    with rasterio.open(hls_red_band_files[0]) as ds:
        profile = ds.profile
        profile['height'] = red_merged.shape[-2]
        profile['width'] = red_merged.shape[-1]
    
    with rasterio.open(final_download_path / 'hls_red.tif', 'w', **profile) as ds:
        ds.write(red_merged)

    with rasterio.open(hls_nir_band_files[0]) as ds:
        profile = ds.profile
        profile['height'] = red_merged.shape[-2]
        profile['width'] = red_merged.shape[-1]
    
    with rasterio.open(final_download_path / 'hls_nir.tif', 'w', **profile) as ds:
        ds.write(nir_merged)

else:
    hls_red_band_files[0].replace(hls_red_band_files[0].parent.parent / hls_red_band_files[0].name)
    hls_nir_band_files[0].replace(hls_nir_band_files[0].parent.parent / hls_nir_band_files[0].name)

In [28]:
# Delete temporary files and paths
shutil.rmtree(Path("../data/common_files/tmp"))